# Tutorial, Part I: Candidate Extraction

In this example, we'll be writing an application to extract **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate chemical-disease (C-D) relation mentions_ as either true or false.  To do this, we first need a set of such candidates- in this notebook, we'll use `DDLite` utilities to extract these candidates.

## Loading the Corpus

First, we will load and pre-process the corpus, storing it for convenience in a `Corpus` object

### Configuring a document parser

We'll start by defining a `DocParser` class to read in Pubmed abstracts from [Pubtator]([Pubtator](http://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi)), where they are stored along with "gold" (i.e. hand-annotated by experts) *chemical* and *disease mention* annotations. We'll use the `XMLDocParser` class, which allows us to use [XPath queries](https://en.wikipedia.org/wiki/XPath) to specify the relevant sections of the XML format.

_Note that we are newline-concatenating text from the title and abstract together for simplicity, but if we wanted to, we could easily extend the `DocParser` classes to preserve information about document structure._

In [1]:
from ddlite_parser import XMLDocParser
xml_parser = XMLDocParser(
    path='data/CDR_DevelopmentSet.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()',
    keep_xml_tree=True)

### Selecting a sentence parser

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.  Here we use the default `SentenceParser` class.

In [2]:
from ddlite_parser import SentenceParser
sent_parser = SentenceParser()

### Pre-processing & loading the corpus

Finally, we'll put this all together using a `Corpus` object, which will execute the parsers and store the results as an iterator:

In [3]:
from ddlite_parser import Corpus
%time corpus = Corpus(xml_parser, sent_parser)

Parsing documents...
Parsing sentences...
CPU times: user 4.43 s, sys: 129 ms, total: 4.55 s
Wall time: 28 s


In [4]:
doc  = corpus.get_docs()[2]
print doc

Document(id='1445986', file='CDR_DevelopmentSet.xml', text="Cefotetan-induced immune hemolytic anemia.\nImmune hemolytic anemia due to a drug-adsorption mechanism has been described primarily in patients receiving penicillins and first-generation cephalosporins. We describe a patient who developed anemia while receiving intravenous cefotetan. Cefotetan-dependent antibodies were detected in the patient's serum and in an eluate prepared from his red blood cells. The eluate also reacted weakly with red blood cells in the absence of cefotetan, suggesting the concomitant formation of warm-reactive autoantibodies. These observations, in conjunction with clinical and laboratory evidence of extravascular hemolysis, are consistent with drug-induced hemolytic anemia, possibly involving both drug-adsorption and autoantibody formation mechanisms. This case emphasizes the need for increased awareness of hemolytic reactions to all cephalosporins.", attribs={'root': <Element document at 0x10a947e60>}

In [5]:
sent = corpus.get_sentences_in(doc.id)[0]
print sent

Sentence(id='1445986-0', words=[u'Cefotetan-induced', u'immune', u'hemolytic', u'anemia', u'.'], lemmas=[u'cefotetan-induced', u'immune', u'hemolytic', u'anemia', u'.'], poses=[u'JJ', u'JJ', u'JJ', u'NN', u'.'], dep_parents=[4, 4, 4, 0, 4], dep_labels=[u'amod', u'amod', u'amod', u'ROOT', u'punct'], sent_id=0, doc_id='1445986', text=u'Cefotetan-induced immune hemolytic anemia.', char_offsets=[0, 18, 25, 35, 41], doc_name='CDR_DevelopmentSet.xml')


## Writing a basic candidate extractor

Next, we'll write a basic function to extract **candidate disease mentions** from the corpus.  For this first attempt, we'll just write a function that checks for matches against a list (or _"dictionary"_) of disease phrases, constructed using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`).

We'll do this using a `CandidateSpace` object--which defines the basic candidates we consider, in this case n-grams up to a certain length--and a `Matcher` object, which filters this candidate space down.

In [6]:
from load_dictionaries import load_disease_dictionary
from ddlite_candidates import Ngrams
from ddlite_matchers import DictionaryMatch

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

# Define a candidate space
ngrams = Ngrams(n_max=3)

# Define a matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Loaded 507899 disease phrases!


Note that we set `longest_match_only=False`, which means that we _will_ consider subsequences of phrases that match our dictionary.

The `Ngrams` operator is applied over our `Sentence` objects and returns `Ngram` objects, and the `Matcher` then filters these, so we apply our operators over the sentences in the corpus, storing the results in a `Candidates` object for convenience:

In [7]:
from ddlite_candidates import Candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.get_candidates()[:5]

Extracting candidates...
CPU times: user 5.08 s, sys: 230 ms, total: 5.31 s
Wall time: 5.15 s


[<Ngram("depression", id=6794356-5:662-671, chars=[662,671], words=[4,4]),
 <Ngram("fulminant hepatitis", id=3411101-5:504-522, chars=[504,522], words=[8,9]),
 <Ngram("examined", id=16132524-4:674-681, chars=[674,681], words=[29,29]),
 <Ngram("hypertension", id=16820346-4:652-663, chars=[652,663], words=[3,3]),
 <Ngram("artery disease", id=15531665-5:829-842, chars=[829,842], words=[16,17])]

## Evaluating our candidate recall on gold annotations

Next, we'll test our _candidate recall_--in other words, how many of the true disease mentions we picked up in our candidate set--using the gold annotations in our dataset.

The XML documents that we loaded using the `XMLDocParser` also contained annotations (this is why we kept the full xml tree using `keep_xml_tree=True`).  We'll load these annotations and map them to `Ngram` objects over our parsed sentences, that way we can easily compare our extracted candidate set with the gold annotations.  The code is fairly simple (see `tutorial/util.py`); note that we filter to only keep _disease_ annotations, and that the candidates should be uniquely identified by their `id` attribute:

In [11]:
from utils import collect_pubtator_annotations
gold = []
for doc, sents in corpus:
    gold += [a for a in collect_pubtator_annotations(doc, sents) if a.metadata['type'] == 'Disease']
gold = frozenset(gold)

Now, we have a set of gold annotations of the same type as our candidates (`Ngram`), and can use set operations (where candidate objects are hashed by their `id` attribute), e.g.:

In [12]:
len(gold.intersection(c.get_candidates()))

3039

For convenience, we'll use a basic helper method of the `Candidates` object:

In [13]:
c.gold_stats(gold)

# of gold annotations	= 4244
# of candidates		= 6944
Candidate recall	= 0.716
Candidate precision	= 0.438


We note that our focus in this stage is on **acheiving high candidate recall, without considering an impractically large candidate set**.  Our main focus after this stage will be on training a classifier to select which candidates are true; this will raise precision while hopefully keeping recall high.  _Note however that candidate recall is an upper bound for the recall of this classifier!_

So, we have some work to do.

## Using the `Viewer` to inspect data

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

To start, we'll assemble a random set of all the sentences where there are gold annotations _not in our candidate set_, i.e. where we missed something, and then inspect these in the `Viewer`:

In [17]:
# Index the gold annotations by sentence id
gold_by_sid = defaultdict(list)
for g in gold:
    gold_by_sid[g.sent_id].append(g)

# Get sentences
view_sents = [s for s in corpus.get_sentences() if len(c.get_candidates_in(s.id)) < len(gold_by_sid[s.id])]
shuffle(view_sents)
view_sents = view_sents[:50]

Now, we instantiate and render the `Viewer` object; note we're being a bit sloppy, passing in _all_ the candidates and gold labels, but the `Viewer` object will take care of indexing them by sentence, and will only render the sentences we pass in:

In [18]:
from ddlite_viewer import SentenceNgramViewer
sv = SentenceNgramViewer(view_sents, c.get_candidates(), gold=gold)
sv.render(n_per_page=3, height=225)

Viewer 
 
 
 
 
 
 DISCUSSION: Swallowing-induced atrial tachyarrhythmia ( SIAT ) is a rare phenomenon. 
 
 We assessed force coordination of the hand in Parkinson's disease and its relationship to motor complications of levodopa therapy, particularly to levodopa-induced dyskinesias ( LID ). 
 
 In single dose studies, DH , MD , and VSD were induced on GDs 9 and 10. 
 
 
 
 
 
 Atropine converted the dystonic movements into chorea . 
 
 Neutrophils from 14 ALF patients were stimulated via the complement receptors using zymosan opsonized with ALF or control serum. 
 
 In 40% of treated patients, an arthralgia - myalgia syndrome develops. 
 
 
 
 
 
 We report on an infant with multiple cranial-nerve involvement attributable to brainstem dysgenesis , born to a cocaine-addicted mother. 
 
 Puromycin aminonucleoside-induced renal dysfunction and hyperlipidemia were also suppressed. 
 
 Together with previously reported cases, our patient suggests that tic-like symptoms might occur in certain vulnerable individuals during treatment with atypical antipsychotics such as quetiapine, clozapine, or amisulpride. 
 
 
 
 
 
 A muscle biopsy in this patient showed loss of thick, myosin filaments . 
 
 Questions remain about the optimal dose of ribavirin and the incidence of RIHA in a real-world population. 
 
 Whereas cells in a wider area of the GPi may be implicated in parkinsonism , the ventral GPi seems to be crucial for the manifestation of LID . 
 
 
 
 
 
 Based on this model and on Bayes' theorem, the predictive accuracy of identifying " nephrotoxic " patients increased from 0.17 to 0.39. 
 
 Severity of peak-dose dyskinesias was strongly correlated with grip force in ON-state (r = 0.79 with peak force, P < 0.01). 
 
 IBPB may cause transient auditory dysfunction in the ipsilateral ear, possibly via an effect on sympathetic innervation. 
 
 
 
 
 
 RESULTS: Of 60 epileptic women with periconceptional folic acid (0.8 mg)-containing multivitamin supplementation, no one developed epilepsy -related side effects during the periconception period. 
 
 Several routine indicators of nephrotoxicity , the enzymes alkaline phosphatase and N-acetyl-beta-glucosaminidase, and the molecular weight of protein excretion were determined on urine samples. 
 
 St. Anthony's fire , then and now: a case report and historical review. 
 
 
 
 
 
 From the analysis of these cases, two distinct forms of carbamazepine-associated cardiac dysfunction emerged. 
 
 We describe herein 3 patients who developed rosacea -like dermatitis eruptions while using 0.03% or 0.1% tacrolimus ointment for facial dermatitis . 
 
 However, fluid attenuated inversion recovery (FLAIR) sequences frequently revealed persistent white matter abnormalities . 
 
 
 
 
 
 RESULTS: Impairment of cognitive function was observed 6 to 8 hours after administration of haloperidol on Day 2 but was not evident 23 hours after dosing. 
 
 The protective role of mangiferin was analyzed by triphenyl tetrazolium chloride (TTC) test used for macroscopic enzyme mapping assay of the ischemic myocardium . 
 
 OBJECTIVE: To achieve absolute risk estimates of venous thromboembolism ( VTE ) among women on cyproterone acetate plus ethinylestradiol (CPA/EE), and among women on combined oral contraceptives (COCs). 
 
 
 
 
 
 Four hours after MCAO , the rats were killed and the brains harvested. 
 
 In the IPRK phenylephrine and acetylcholine related responses were significantly impaired in the ADR- nephropathy group. 
 
 This was associated with a reduced activation state of AKT, as well as an increased bax/bcl2 ratio in Western blots, indicating cardiac apoptosis . 
 
 
 
 
 
 There have been concerns that the risk of cardiovascular thrombotic events may be higher with cyclooxygenase (COX)-2-specific inhibitors than nonselective nonsteroidal antiinflammatory drugs (NSAIDs). 
 
 Placebo-level incidence of extrapyramidal symptoms ( EPS ) with quetiapine in controlled studies of patients with bipolar mania . 
 

## Composing a better candidate extractor

Let's try to increase our candidate recall using `Matcher` operators.  We'll see in particular that they are compositional; noting that we are missing all of the acronyms, let's start with the `Union` operator, to integrate a dictionary for this:

In [21]:
from ddlite_matchers import Union
from load_dictionaries import load_acronym_dictionary

# Load the disease phrase dictionary
acronyms = load_acronym_dictionary()
print "Loaded %s acronyms!" % len(acronyms)

# Define a new matcher
matcher = Union(
    DictionaryMatch(d=diseases, longest_match_only=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Loaded 36904 acronyms!
Extracting candidates...
CPU times: user 5.54 s, sys: 156 ms, total: 5.7 s
Wall time: 5.59 s
# of gold annotations	= 4244
# of candidates		= 7530
Candidate recall	= 0.747
Candidate precision	= 0.421


Next, we try using the `Concat` and `RegexMatch` operators to find candidate mentions composed of an _adjective followed by a term matching our diseases dictionary_.  Note in particular that we set `left_required=False` so that exact matches to our dictionary (with no adjective prepended) will still work:

In [22]:
from ddlite_matchers import Concat, RegexMatchEach
matcher = Union(
    Concat(
        RegexMatchEach(rgx=r'JJ*', attrib='poses'),
        DictionaryMatch(d=diseases, longest_match_only=False),
        left_required=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
%time c = Candidates(ngrams, matcher, corpus.get_sentences())
c.gold_stats(gold)

Extracting candidates...
CPU times: user 15.9 s, sys: 697 ms, total: 16.6 s
Wall time: 16.1 s
# of gold annotations	= 4244
# of candidates		= 9302
Candidate recall	= 0.789
Candidate precision	= 0.360


### More coming here...

We've increased the candidate recall (on the development set) by 7.3% using some simple compositional `Matcher` operators.  We'll be adding more here soon!

## TO-DO: Show how to connect to rest of pipeline!